# Knowledge Acquisition of XAI Recommmender (XAIR)

## For editing or inserting knowledge 

<p style="color: red; font-weight: bold; font-size: 16px; line-height: 32px">
Please note that existing XAI methods/criteria will be overwritten if their name/label is the same as an existing one. 
</p>

So if you want to edit an existing XAI method/criterion, be sure to use the same name and label.


If there are any questions, please don't hestitate to contact Verena Barth (verena.barth@viadee.de)

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

import os
from ipywidgets import Layout, widgets, Box, Textarea,Dropdown, Label, IntSlider, Text, RadioButtons, IntText
from Utils import get_config, ANTE_CONFIG_FILE, get_ante_config, create_exclusion_criteria, create_criteria, add_method
import pandas as pd
import numpy as np
import json
from IPython.display import display

from resources.config.antecedent_config import english_ratings, input_types, sections, crit_types
#from resources.config.antecedent_config import antecedents as c


config = get_config()
ante_config = get_ante_config()

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

## Add exclusion criteria (only boolean)

### You have to provide the criteria information listed below

* Name of the criteria
* Label to be displayed
* Text for helping user to understand criteria
* Section to display input field
* Rating for all available XAI method (listed below)

<b>Attention: Formulate Boolean criteria, that application of the method is possible when input variable >= Rating of method in dataframe</b>

<p style="color: red; font-weight: bold; font-size: 18px; line-height: 32px">
Please fill out the form and execute the cell below! </p>

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

form_items = [
    Box([Label(value='Criterion name'), 
         Text(
            placeholder='my_criterion')],
        layout=form_item_layout),
     Box([Label(value='Criterion label'), 
         Text(
            placeholder='My Criterion')],
        layout=form_item_layout),
    Box([Label(value='Help text'),
         Textarea(placeholder="A text to help the user")], 
        layout=form_item_layout),
    Box([Label(value='Which section does it belong to?'),
         Dropdown(options=sections)], 
        layout=form_item_layout)
]

# rate all methods
methods = pd.read_csv(config.resource_files.rating_bool, index_col=[0]).columns.values
for m in methods:
    form_items.append(Box([Label(value=f"Rating for method {m}"),
         RadioButtons(options=["Yes", "No"],layout=form_item_layout)], 
        layout=form_item_layout))

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    width='80%',
))
form

### Please execute cell below to add exclusion criterion

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

rating_idx = len(form.children) - len(methods) 
crit_inputs = []
temp_rating = []
for idx,c in enumerate(form.children):
    assert c.children[1].value, f"Input field of '{c.children[0].value}' should not be empty"
    if idx < rating_idx:
        crit_inputs.append(c.children[1].value)
    else:
        rating = 1 if c.children[1].value == "Yes" else 0
        temp_rating.append(rating)
crit_inputs.append(temp_rating)

def add_exclusion_crit(crit_name, crit_label, help_txt,section,ratings):
    ante_config[crit_name] = create_exclusion_criteria(crit_name, crit_label, help_txt,section,ratings)
    # save changes to antecedent config
    with open(ANTE_CONFIG_FILE, 'w') as fp:
        json.dump(ante_config, fp)
        
        

add_exclusion_crit(*crit_inputs)
print("You have successfully added your exclusion criteria.")

crit_inputs = []
temp_rating = []

<p style="color: red; font-weight: bold; font-size: 20px; line-height: 32px">
To apply the changes, please set "reload=True" for the next build of the XAIR!</p>

<br/>
<br/>
<br/>

## Add "normal" criteria

### Please provide the criteria information listed below

* If criteria is should cause a different XAI method rating, select crit_type = RATING_CRIT
* If criteria only 'activates' othe rrules, select crit_type = RULE_ACTIVATION_CRITERIA
* If the recommender should not use it for determining the suitability of a method, select crit_type = OTHER

<p style="color: red; font-weight: bold; font-size: 15px; line-height: 32px">
If it is a FOI specific criterion....
    </p>

* Add suffix "_foi", e.g. "yourcriterion_foi"
* Add custom rule to the file <b>xai_xps/src/resources/config/custom_rules.json</b> under <b>use_ratings_with_prereq</b> with the following syntax:

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

form_fuzzy_items = [
    Box([Label(value='Criterion name'), 
         Text(
            placeholder='my_criterion')],
        layout=form_item_layout),
     Box([Label(value='Criterion label'), 
         Text(
            placeholder='My Criterion')],
        layout=form_item_layout),
    
    Box([Label(value='Help text'),
         Textarea(placeholder="A text to help the user")], 
        layout=form_item_layout),
    
    Label(value="Now please select the criterion type."),
    Label(value="-  If criterion should cause a different XAI method rating, select crit_type = RATING_CRIT."),
    Label(value="-  If criterion only 'activates' othe rules, select crit_type = RULE_ACTIVATION_CRITERIA."),
    Label(value="-  If the recommender should not use it for determining the suitability of a method, select crit_type = OTHER"),
     Box([Label(value='Criterion type', style={"font_weight":"bold"}),
         Dropdown(options=crit_types)], 
        layout=form_item_layout),
    

    
    Label(value="If criterion has a standalone impact on the ratings of the XAI methods, set to 'Yes'"),
    Label(value="If it's just necessary in combination with other critera, set to 'No'"),
    Box([Label(value="Standalone impact"),
         RadioButtons(options=["Yes", "No"],layout=form_item_layout)], 
        layout=form_item_layout),
    
    Label(value="If criterion has no impact at all (e.g. model_name, list of FOI), set to 'Yes'"),
    Box([Label(value="Disable processing"),
         RadioButtons(options=["Yes", "No"],layout=form_item_layout)], 
        layout=form_item_layout),
    
    Box([Label(value='Which section does it belong to?'),
         Dropdown(options=sections)], 
        layout=form_item_layout),
    
    Box([Label(value='Input type'),
     Dropdown(options=input_types)], 
    layout=form_item_layout),
    
    Label(value="Specify the number of nuances you want the criterion to have"),
    Box([Label(value='Number of ratings'),
     Dropdown(options=["2", "3", "5"])], 
    layout=form_item_layout),
    
    Label(value="Rating labels (number of ratings = number of fuzzy memberships!)"),
    Box([Label(value='Rating labels'),
             RadioButtons(
            options=[
                *english_ratings.items(),
                'Use custom...'
            ],
            layout={'width': 'max-content'}
        )
        ], 
        layout=form_item_layout),
    
    Box([Label(value='Custom rating labels (Provide as dict)'), 
         Text(
             value="{'L': 'low', 'M': 'medium', 'H': 'high'}",
            placeholder="{'L': 'low', 'M': 'medium', 'H': 'high'}")],
        layout=form_item_layout),
    
    
    
    Box([Label(value='Select membership type'),
     Dropdown(options=["Triangular", "Trapeziodal"])], 
    layout=form_item_layout)
]

form_fuzzy = Box(form_fuzzy_items, layout=Layout(
    display='flex',
    flex_flow='column',
    width='80%',
))
display(form_fuzzy)

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

# validate form 1

form_fuzzy_vals = [c.children[1].value for c in form_fuzzy.children if type(c) != Label]

assert int(form_fuzzy_vals[-4]) == len(form_fuzzy_vals[-3][1]),f"Number of rating labels ({len(form_fuzzy_vals[-3][1])}) != Number of rating ({int(form_fuzzy_vals[-4])})"

for i in form_fuzzy_vals:
    assert i != "", "No field should be empty"

try:
    rating = dict(form_fuzzy_vals[-3][1])
except:
    # use custom rating
    #assert dict(form_fuzzy_vals[-2]), f"Rating provided is not valid ('{form_fuzzy_vals[-2]}'). Please insert python list."
    rating = json.loads(form_fuzzy_vals[-2].replace("'",'"'))
    assert isinstance(rating,dict), f"Rating provided is not valid ('{form_fuzzy_vals[-2]}'). Please insert python dict."
    assert len(rating) == int(form_fuzzy_vals[-4]), f"Number of rating labels ({len(rating)}) != Number of rating ({int(form_fuzzy_vals[-4])})"
    
    
form_fuzzy_vals = [c.children[1].value for c in form_fuzzy.children if type(c) != Label]

if int(form_fuzzy_vals[-4]) == 2:
    universe = config.fuzzy.universes.u_bool
    mem_funcs = config.fuzzy.mem_funcs.bool
    frontend_rating_txt = english_ratings["bool"]
    dtypes = {
            "fuzzy": "bool",
            "crisp": "bool"
        }
else:
    form_fuzzy_items_2 = [
        Label(value="Your new criterion is not a boolean criterion."),
        
        Box([Label(value="Which datatype is it, when it's input is exact?"),
        Dropdown(options=["str", "float", "int", "list"])], 
        layout=form_item_layout),
        
        Box([Label(value="Which datatype is it, when it's input is fuzzy?"),
        Dropdown(options=["str", "float", "int", "list"])], 
        layout=form_item_layout),
        Label(value="(Please select 'list' if your input type mentioned above is 'list')"),
        
        Box([Label(value='Initial value (set in Frontend)'), 
        IntText(
            value=0,
        )],
        layout=form_item_layout),  
        Box([Label(value='Min value'), 
        IntText(
            value=0,
        )],
        layout=form_item_layout), 
        Box([Label(value='Max value'), 
        IntText(
            value=10,
        )],
        layout=form_item_layout), 
        
        Label(value="Please specify the universe and the number and form of the membership functions. You can use exisiting functions or specify new ones."),
        Label(value="There are some predefined triangular and trapeziodal membership functions."),
        
        Box([Label(value='Universes'),
             RadioButtons(
                options=[
                    *config.fuzzy.universes.__dict__.values(),
                    'Use custom...'
                ],
                layout={'width': 'max-content'}
            )], 
                 layout=form_item_layout),
         Box([Label(value='Custom Universe (provide list)'), 
         Text(
            placeholder='[1,2,3]')],
        layout=form_item_layout)
    ]
    
    if form_fuzzy.children[-1].children[1].value == "Triangular":
        form_fuzzy_items_2.append(
        Box([Label(value='Membership functions (triangular)'),
             RadioButtons(
            options=[
                *config.fuzzy.mem_funcs.tri.__dict__.values(),
                'Use custom...'
            ],
            layout={'width': 'max-content'}
        )
        ], 
                 layout=form_item_layout))
        
        form_fuzzy_items_2.append(Box([Label(value='Custom triangular membership function (provide list)'), 
         Text(
            placeholder='[[0, 0, 5], [0, 5, 10], [5, 10, 10]]')],
        layout=form_item_layout))
    else:
        form_fuzzy_items_2.append(
             Box([Label(value='Membership functions (trapeziodal)'),
        RadioButtons(
            options=[
                *config.fuzzy.mem_funcs.trap.__dict__.values(),
                'Use custom...'
            ],
            layout={'width': 'max-content'}
       )
        ], 
                 layout=form_item_layout))
        
        form_fuzzy_items_2.append(Box([Label(value='Custom triangular membership function (provide list)'), 
         Text(
            placeholder='[[0, 0, 2, 3], [0, 0, 4, 5], [4, 5, 6, 7], [6, 7, 10, 10], [8, 9, 10, 10]]')],
        layout=form_item_layout))
            

    form_fuzzy_2 = Box(form_fuzzy_items_2, layout=Layout(
        display='flex',
        flex_flow='column',
        width='80%',
    ))
    display(form_fuzzy_2)    

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

# validate form 2 if available

if int(form_fuzzy_vals[-4]) != 2:
    form_fuzzy_2_vals = [c.children[1].value for c in form_fuzzy_2.children if type(c) != Label]
    
    dtypes = {
        "fuzzy": form_fuzzy_2_vals[1],
        "crisp": form_fuzzy_2_vals[0]
    }
    
    f_u = form_fuzzy_2_vals[5]
    f_uc = form_fuzzy_2_vals[6]
    f_m = form_fuzzy_2_vals[-2]
    f_mc = form_fuzzy_2_vals[-1]
    
    f_init = form_fuzzy_2_vals[2]
    f_min = form_fuzzy_2_vals[3]
    f_max = form_fuzzy_2_vals[4]

    if f_u == "Use custom...":
        assert len(list(f_uc)) > 0, f"Universe provided is not valid ('{f_uc}'). Please insert python list."
        u = json.loads(f_uc)
    else: 
        u = list(f_u)

    if f_m == "Use custom...":
        assert len(list(f_mc)) > 0, f"Universe provided is not valid ('{f_mc}'). Please insert python list."
        mem_func = json.loads(f_mc)
    else: 
        mem_func = list(f_m)


    # check validity
    assert u[0] <= mem_func[0][0] and u[-1] >= mem_func[-1][-1] , "Universe must cover membership functions!"
    assert len(rating) == len(mem_func), f"Number of memberships ({len(mem_func)}) must be equal to number of rating ({len(rating)})!" 

    assert u[0] <= int(f_init) <=  u[-1], "Initial value must be within universe!"
    assert f_min >=  u[0], f"Min value ({f_min}) must be within universe! (>= {u[0]})"
    assert f_max <=  u[-1], f"Max value ({f_max}) must be within universe! ( <= {u[-1]})"

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

def add_criteria(
        name, 
        label, 
        help_txt,
        crit_type, 
        standalone_impact, 
        disable_processing, 
        section,
        input_type, 
        rating, #num_rating = len(rating)
        universe,
        mem_funcs,
        dtypes,
        init_value=None, 
        max_value=None, 
        min_value=None, 
        df_rating=None):
    
    standalone = 1 if standalone_impact == "Yes" else 0
    if standalone:
        assert df_rating is not None, "Please provide a rating dataframe"
        assert not df_rating.isnull().values.any(), "Please replace NaN values from rating DataFrame with '-'."
    
    ante_config[name] = create_criteria(
        label, 
        help_txt,
        crit_type, 
        standalone, 
        1 if disable_processing == "Yes" else 0, 
        section,
        input_type, 
        rating, #num_rating = len(rating)
        universe,
        mem_funcs,
        dtypes,
        init_value, 
        max_value, 
        min_value)

    #print(ante_config[name])
    # save changes to antecedent config
    with open(ANTE_CONFIG_FILE, 'w') as fp:
        json.dump(ante_config, fp)
        
    # if standalone impact, add to rating df
    if standalone:
        rating_df =  pd.read_csv(config.resource_files.rating_fuzzy, index_col=[0, 1])

        index =  [r for r in config.ratings if len(r) == len(rating)][0]
        for i in index:
            rating_df.loc[(label,i),:] = df_rating.loc[i]

        rating_df.to_csv(config.resource_files.rating_fuzzy)

        return rating_df

## If criterion DOES NOT HAVE a standalone impact on XAI method suitabilities:

### Add to custom rules

Add custom rule to the file <b>xai_xps/src/resources/config/custom_rules.json</b>.

If you need a prerequisite to activate criterion, add it to <b>use_ratings_with_prereq</b> like that:

To define new custom rules add it to <b>custom_rules</b> with the following syntax:

In [ ]:
inputs

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

if form_fuzzy_vals[4] == "No":
    inputs = form_fuzzy_vals[:-4]
    inputs.extend([rating, u, mem_func, dtypes])
    print(f"You have successfully added your fuzzy criteria '{form_fuzzy_vals[1]}'")
    add_criteria(*inputs)

If your criterion has no standalone impact, please stop filling out the following fields.


## If criterion has a standalone impact on XAI method suitabilities:

### Rate all methods for all fuzzy terms

* Add different weights by using '%', <br/> e.g. rating of 'L' with weight of 60%: 'L%60'
* If criteria are irrelevant for method suitability, add "-"

### Please make sure all ratings are valid

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

df_rating = pd.DataFrame(index = [r for r in config.ratings if len(r) == len(rating)][0], columns = pd.read_csv(config.resource_files.rating_fuzzy, index_col=[0, 1]).columns)
df_rating

In [ ]:
df_rating.loc["L"] = "L"
df_rating.loc["M"] = "M"
df_rating.loc["H"] = "H"

In [ ]:
# TODO 
# PLEASE INSERT YOUR VALUES HERE ...

#For rating criteria value of 'L' for 'ALE':
#rating of 'L' with weight of 60%: 'L%60'


df_rating.loc["L", "ALE"] = "L%60"
df_rating.loc["L", "PFI"] = "-"

# insert whole row
df_rating.loc["M"] = ["L", "M", "H", "-", "L%0.6", "H"]

#### to be continued....

In [ ]:
df_rating

### Please check if rating entries are correct

In [ ]:
df_rating

### Finally, save rating DataFrame and rating config!

In [ ]:
get_ante_config()["corr"]["frontend"]["max"]

In [ ]:
antecedents = np.load(config.resource_files.antecedents, allow_pickle=True).item()
antecedents

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

inputs = form_fuzzy_vals[:-4]
inputs.extend([rating, u, mem_func, dtypes, *form_fuzzy_2_vals[2:5],df_rating])
print(f"You have successfully added your fuzzy criteria '{form_fuzzy_vals[1]}'")
add_criteria(*inputs)

<p style="color: red; font-weight: bold; font-size: 20px; line-height: 32px">
To apply the changes, please set "reload=True" for the next build of the XAIR!</p>

<br/>
<br/>
<br/>


## Add XAI Method

### Steps to take

* Insert method characteristics (incl. exclusion criteria)
* Rate method regarding fuzzy criteria
* Add method information to be displayed in web application

### Insert method characteristics (incl. exclusion criteria)

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

form_items_method = [
    Box([Label(value='Method name'), 
         Text(
            placeholder='my_method')],
        layout=form_item_layout),
    Box([Label(value='Method label'), 
         Text(
            placeholder='My Method')],
        layout=form_item_layout),
    
    Box([Label(value="Is it a visualization method?"),
         RadioButtons(options=["Yes", "No"],layout=form_item_layout)], 
        layout=form_item_layout),
    Box([Label(value="Is the explanation global?"),
         RadioButtons(options=["Yes", "No"],layout=form_item_layout)], 
        layout=form_item_layout),
    Box([Label(value="Is the explanation local?"),
         RadioButtons(options=["Yes", "No"],layout=form_item_layout)], 
        layout=form_item_layout)
]

# rate all methods
exclusion_crits = list(pd.read_csv(config.resource_files.rating_bool, index_col=[0]).index)

for e in exclusion_crits:
    form_items_method.append(Box([Label(value=e),
         RadioButtons(options=["Yes", "No"],layout=form_item_layout)], 
        layout=form_item_layout))

form_method = Box(form_items_method, layout=Layout(
    display='flex',
    flex_flow='column',
    width='80%',
))
form_method

### Now rate the method for all fuzzy terms

* Add different weights by using '%', <br/> e.g. rating of 'L' with weight of 60%: 'L%60'
* If criteria are irrelevant for method suitability, add "-"



### Please make sure all ratings are valid

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

# create rating df

method_label = form_items_method[1].children[1].value
df_rating_method = pd.DataFrame(index = pd.read_csv(config.resource_files.rating_fuzzy, index_col=[0, 1]).index, columns = [method_label])
# add fuzzy ratings from boolean input above
df_rating_method.loc["Local Scope", "False"] = "-"
df_rating_method.loc["Global Scope", "False"] = "-"
df_rating_method.loc["Global Scope", "True"] = "VH" if form_items_method[3].children[1].value == "Yes" else "-"
df_rating_method.loc["Local Scope", "True"] = "VH" if form_items_method[4].children[1].value == "Yes" else "-"
df_rating_method

In [ ]:
df_rating_method[method_label] = "-" 
df_rating_method.loc[('Correlation','L'), method_label] = "H"
df_rating_method.loc[('Correlation FOI','L'), method_label] = "H"

# TODO
# continue rating .....
df_rating_method

In [ ]:
############################## Please execute cell ##############################
################################## Do not alter #################################

def add_fuzzy_method(name,label,visualization, exclusion_ratings, df):
    assert not df.isnull().values.any(), "Please replace NaN values from rating DataFrame with '-'."
    vis = 1 if visualization=="Yes" else 0
    add_method(name,label,vis, exclusion_ratings, df)
    
    
exclusion_ratings = [1 if c.children[1].value == "Yes" else 0 for c in form_method.children[5:] if type(c.children[1]) != Text]
_ = add_fuzzy_method(*[c.children[1].value for c in form_method.children[:3]], exclusion_ratings, df_rating_method)

### Add method information to be displayed in web application

In the web application frontend (project "xai-recommender-frontend"), please add a page called <method_name>.json in the following directory: 

<b>xai-recommender-frontend/static/methods</b>

A schema for the JSON validation of the method file can be found under "xai-recommender-frontend\static\jsonschema_methods.json". It can be validated for example via  https://www.jsonschemavalidator.net/


<p style="color: red; font-weight: bold; font-size: 20px; line-height: 32px">
To apply the changes, please set "reload=True" for the next build of the XAIR!</p>
